In [3]:
import numpy as np
import requests

In [282]:
def llm(question, prompts=None, examples=None):
    response = requests.post(
        "http://172.19.8.33:8080/generate/",
        json={
            "text": f"{question}",
            "generate_tokens_limit": 2048 - 1305,
            "top_p": 1,
            "top_k": 0,
            "temperature": 0.5,
            "repetition_penalty": 1.5,
            "prompt": prompts,
            "examples": examples
            },
        headers={
            "Content-Type": "application/json"
            }
        )
    return response.json()

In [276]:
seq = np.random.normal(loc=0, scale=1, size=5).round(2).tolist()
question = f"Are there any outliers in the sequence {seq}?"


In [283]:
import outlier_env


env = outlier_env.OutlierEnv()


def cleaning_peter(question, prompts=None, examples=None):#(idx=None, prompt=prompt, to_print=True):
    question = examples + " \nQuestion: " + question
    
    # for i in range(1, 8):
    thought_action = llm(question=question + f"Thought {i}:", prompts=prompts, examples=examples)["completion"]
    print("-------------THOUGHT ACTION BEFORE-------------")
    print(thought_action)
    print("^^^^^^^^^^^^^THOUGHT ACTION BEFORE^^^^^^^^^^^^^")
    thought_action = thought_action[thought_action.find(f'Thought {i}'):thought_action.find(f'\nObservation {i}')]
    print("-------------THOUGHT ACTION AFTER-------------")
    print(thought_action)
    print("^^^^^^^^^^^^^THOUGHT ACTION AFTER^^^^^^^^^^^^^")
    #     try:
    #         thought, action = thought_action.strip().split(f"\nAction {i}: ")
    #     except:
    #         print('ohh...', thought_action)
    #         n_badcalls += 1
    #         n_calls += 1
    #         thought = thought_action.strip().split('\n')[0]
    #         action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
    #     obs, r, done, info = step(env, action[0].lower() + action[1:])
    #     obs = obs.replace('\\n', '')
    #     step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
    #     prompt += step_str
    #     if to_print:
    #         print(step_str)
    #     if done:
    #         break
    # if not done:
    #     obs, r, done, info = step(env, "finish[]")
    # if to_print:
    #     print(info, '\n')
    # info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    # return r, info

In [278]:
print(question)

Are there any outliers in the sequence [-0.17, 1.45, 1.56, 1.98, -0.08]?


In [284]:
cleaning_peter(question=question, examples=examples)

-------------THOUGHT ACTION BEFORE-------------

                
Question: Are there any outliers in the sequence [-1.7, 0.2, 1.7, 0.5, -1.1]?
                
Thought 1: I need to find outliers in the sequence.
                
Action 1: outlier_detection_step[-1.7, 0.2, 1.7, 0.5, -1.1]
                
Observation 1: The outliers are {0: False, 1: False, 2: False, 3: False, 4: False}
                
Thought 2: The entries [] are the outliers in the sequence
                
Action 2: finish[]
                
                
Question: Are there any outliers in the sequence [-2.2, 1.8, -0.4, 0.6, -0.9]?
                
Thought 1: I need to find outliers in the sequence.
                
Action 1: outlier_detection_step[-2.2, 1.8, -0.4, 0.6, -0.9]
                
Observation 1: The outliers are {0: False, 1: False, 2: False, 3: False, 4: False}
                
Thought 2: The entries [] are the outliers in the sequence
                
Action 2: finish[]
                
         

___
___
___
# Generating examples

In [204]:
def outlier_detection_step(sequence: list[float]) -> dict[int, bool]:
    """Given a list of numbers, return a dictionary with the indices of the outliers and a boolean value indicating whether they are
    outliers or not"""
    mean = np.mean(sequence)
    sd = np.std(sequence)
    
    outliers = {idx: (abs(value - mean) > 3 * sd) for idx, value in enumerate(sequence)}
    only_outliers = [idx for idx, value in outliers.items() if value == True]
    
    return outliers, only_outliers

In [271]:
examples = ""
for i in range(1, 6):
    seq = np.random.normal(loc=0, scale=1, size=5).round(1).tolist()
    examples += f"""
                \nQuestion: Are there any outliers in the sequence {seq}?
                \nThought 1: I need to find outliers in the sequence.
                \nAction 1: outlier_detection_step{seq}
                \nObservation 1: The outliers are {outlier_detection_step(seq)[0]}
                \nThought 2: The entries {outlier_detection_step(seq)[1]} are the outliers in the sequence
                \nAction 2: finish{[seq[idx] for idx in outlier_detection_step(seq)[1]]}
                """

In [274]:
2048 - (len(examples) + len(question))

-370

___
___
___

In [183]:
examples = {}
prompts = ""

for _ in range(5):
    numbers = np.random.randint(0, 100, 3).round(2)
    key = f"What is {numbers[0]} + {numbers[1]} - {numbers[2]}?"
    value = f"{calculator(numbers[0], numbers[1], numbers[2])}"
    examples[key] = value
    
    prompts += f"""
    Question: {key}
    calculator({numbers[0]}, {numbers[1]}, {numbers[2]})
    """

In [165]:
examples = {}
prompts = ""

for _ in range(5):
    numbers = np.random.randint(0, 100, 3).round(2)
    key = f"What is {numbers[0]} + {numbers[1]} - {numbers[2]}?"
    value = f"{numbers[0] + numbers[1] - numbers[2]}"
    examples[key] = value
    
    prompts += f"""
                Question: {key}
                Thought 1: First, I need to calculate {numbers[0]} + {numbers[1]}.
                Action 1: {numbers[0]} + {numbers[1]}.
                Observation 1: {numbers[0]} + {numbers[1]} = {numbers[0] + numbers[1]},
                Thought 2: Then, I need to calculate {numbers[0] + numbers[1]} - {numbers[2]}.
                Action 2: {numbers[0] + numbers[1] - numbers[2]}.
                Observation 2: {numbers[0] + numbers[1]} - {numbers[2]} = {numbers[0] + numbers[1] - numbers[2]}, so the answer is {numbers[0] + numbers[1] - numbers[2]}.
                Action 3: I am done.
                """

In [185]:
question = f"What is 10 + 10 - 5?"
question = "INTRUCTION" + question
examples = "Question 1 What is X\nThought 1: ..."

print(llm(question=question, examples=examples)["completion"][len(prompts):])

What is 10 + 10 - 5?
    calculator(10, 10, 5)
    
    Question: What is 55 + 84 - 71?
    calculator(55, 84, 71)
    What is 98 + 8 - 56?
    calculator(98, 8, 56)
    
    Question: What is 10 + 10 - 5?
    calculator(10, 10, 5)
    What is 74 + 38 - 71?
    calculator(74, 38, 71)
    
    Question: What is 55 + 84 - 71?
    calculator(55, 84, 71)
    What is 74 + 38 - 56?
    calculator(74, 38, 56)
    
    Question: What is 10 + 10 - 5?
    calculator(10, 10, 5)
    What is 98 + 8 - 56?
    calculator(98, 8, 56)
    
    Question: What is 55 + 84 - 71?
    calculator(55, 84, 71)
    What is 74 + 38 - 56?
    calculator(74, 38, 56)
    
    Question: What is 55 + 84 - 71?
    calculator(55, 84, 71)
    What is 74 + 38 - 56?
    calculator(74, 38, 56)
    
    Question: What is 14 + 28 - 5?
    calculator(14, 28, 5)
    
    Question: What is 55 + 84 - 71?
    calculator(55, 84, 71)
    What is 74 + 38 - 56?
    calculator(74, 38, 56)
    
    Question: What is 55 + 84 - 71?
    cal